Import Required Libraries

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import chi2_contingency
import seaborn as sns
import re

Retrieve Datasets as Dataframes

In [3]:
df_2024 = pd.read_csv('Dataset Generation (2024) (Responses) - Form Responses 1.csv')
df_fardina = pd.read_csv('Dataset Generation (Fardina) (Responses) - Form Responses 1.csv')
df_max = pd.read_csv('Dataset Generation (Max) (Responses) - Form Responses 1.csv')

In [13]:
df_2024.drop_duplicates(inplace=True)
df_fardina.drop_duplicates(inplace=True)
df_max.drop_duplicates(inplace=True)

df_2024.dropna(inplace=True)
df_fardina.dropna(inplace=True)
df_max.dropna(inplace=True)

short_cols = ['Time', 'Year', 'Age', 'Politics of Parents', 'Politics of Self', 'Spirituality', 'Gender', 'Q1: Ignored Knee Pain', 'Q2: Rejected Aisle Walk', 'Q3: Trust Fund Split 50/50', 'Q4: Schooling Seperate Finances', 'Q5: Lost Cat, No Reward', 'Q6: Surprisig Child Drop-Off', 'Q7: Business Class Over Kids', 'Q8: Single Parent, Four Kids', 'Q9: Split Spouse Child Support', 'Q10: Expensive School, Cheap Career', 'Q11: Ex Wedding Invitation Revoked', 'Q12: Relatives Unapprove Wedding', 'Q13: Pregnant Wife, No Drinks', 'Q14: Rejected Dyed Hair']  

df_fardina_non_priming = pd.DataFrame()
df_fardina_non_priming[df_fardina.columns.to_list()[0:7] + df_fardina.columns.to_list()[8:]] = df_fardina[df_fardina.columns.to_list()[0:7] + df_fardina.columns.to_list()[8:]]

df_fardina['How old are you?'] = df_fardina['How old are you?'].apply(func=lambda x: int(x) if x != '50+' else x)
df_max['How old are you?'] = df_max['How old are you?'].apply(func=lambda x: int(x) if x != '50+' else x)

fardina_cols = df_fardina_non_priming.columns.to_list()
max_cols = df_max.columns.to_list()
fardina_col_mapping = dict(zip(fardina_cols, short_cols))
max_col_mapping = dict(zip(max_cols, short_cols))

df_fardina_final = df_fardina_non_priming.rename(columns=fardina_col_mapping)
df_max_final = df_max.rename(columns=max_col_mapping)

df_2023_final = pd.concat([df_fardina_final, df_max_final])
df_2023_final['Gender'] = df_2023_final['Gender'].apply(func=lambda s: s.replace('Famale', 'Female') if isinstance(s, str) else s)

str_replace = lambda s: s.replace('religious', 'spiritual') if isinstance(s, str) else s
df_2024['How would you rate your religiousness?'] = df_2024['How would you rate your religiousness?'].apply(func=str_replace)
df_2024['How old are you?'] = df_2024['How old are you?'].apply(func=lambda x: int(x) if x != '50+' else x)

d2024_cols = df_2024.columns.to_list()
d2024_col_mapping = dict(zip(d2024_cols, short_cols))
df_2024_final = df_2024.rename(columns=d2024_col_mapping)

df_all = pd.concat([df_2023_final, df_2024_final])

df_fardina_priming_final = df_fardina_final.copy()
df_fardina_priming_final['Compassionate'] = df_fardina['Would you describe yourself as compassionate?']

In [14]:
df_all

,Time,Year,Age,Politics of Parents,Politics of Self,Spirituality,Gender,Q1: Ignored Knee Pain,Q2: Rejected Aisle Walk,Q3: Trust Fund Split 50/50,...,"Q5: Lost Cat, No Reward",Q6: Surprisig Child Drop-Off,Q7: Business Class Over Kids,"Q8: Single Parent, Four Kids",Q9: Split Spouse Child Support,"Q10: Expensive School, Cheap Career",Q11: Ex Wedding Invitation Revoked,Q12: Relatives Unapprove Wedding,"Q13: Pregnant Wife, No Drinks",Q14: Rejected Dyed Hair
0,9/23/2023 18:01:01,Junior,20,Mildly conservative,Mildly conservative,Not spiritual at all,Male,Strongly a jerk,Mildly a jerk,Strongly a jerk,...,Mildly a jerk,Mildly a jerk,Mildly a jerk,Not a jerk,Not a jerk,Strongly a jerk,Not a jerk,Not a jerk,Strongly a jerk,Mildly a jerk
1,9/23/2023 18:01:08,Junior,20,Mildly liberal,Mildly liberal,Not spiritual at all,Male,Mildly a jerk,Mildly a jerk,Strongly a jerk,...,Mildly a jerk,Not a jerk,Strongly a jerk,Not a jerk,Not a jerk,Mildly a jerk,Not a jerk,Not a jerk,Strongly a jerk,Not a jerk
2,9/23/2023 18:08:57,Junior,20,Mildly conservative,Mildly conservative,Somewhat spiritual,Male,Not a jerk,Not a jerk,Mildly a jerk,...,Not a jerk,Mildly a jerk,Mildly a jerk,Mildly a jerk,Not a jerk,Not a jerk,Not a jerk,Mildly a jerk,Strongly a jerk,Strongly a jerk
3,9/23/2023 18:09:57,Senior,22,Mildly liberal,Mildly liberal,Not spiritual at all,Female,Mildly a jerk,Not a jerk,Not a jerk,...,Not a jerk,Not a jerk,Mildly a jerk,Not a jerk,Not a jerk,Strongly a jerk,Not a jerk,Not a jerk,Not a jerk,Mildly a jerk
5,9/23/2023 18:10:13,Other,25,Mildly conservative,Mildly conservative,Not spiritual at all,Male,Mildly a jerk,Not a jerk,Not a jerk,...,Not a jerk,Not a jerk,Not a jerk,Not a jerk,Strongly a jerk,Strongly a jerk,Not a jerk,Mildly a jerk,Not a jerk,Mildly a jerk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,9/18/2024 21:34:10,Graduate Student,21,Don't know / It's complicated,Mildly liberal,Somewhat spiritual,Male,Not a jerk,Strongly a jerk,Not a jerk,...,Not a jerk,Not a jerk,Mildly a jerk,Strongly a jerk,Not a jerk,Not a jerk,Not a jerk,Mildly a jerk,Strongly a jerk,Mildly a jerk
126,9/18/2024 22:40:26,Junior,20,Neutral,Neutral,Somewhat spiritual,Male,Not a jerk,Not a jerk,Not a jerk,...,Strongly a jerk,Mildly a jerk,Mildly a jerk,Not a jerk,Not a jerk,Strongly a jerk,Strongly a jerk,Not a jerk,Strongly a jerk,Strongly a jerk
127,9/18/2024 23:06:25,Senior,21,Mildly conservative,Mildly conservative,Somewhat spiritual,Male,Not a jerk,Not a jerk,Not a jerk,...,Not a jerk,Mildly a jerk,Mildly a jerk,Not a jerk,Strongly a jerk,Mildly a jerk,Not a jerk,Mildly a jerk,Strongly a jerk,Not a jerk
128,9/18/2024 23:26:38,Graduate Student,28,Neutral,Neutral,Somewhat spiritual,Male,Not a jerk,Mildly a jerk,Mildly a jerk,...,Not a jerk,Strongly a jerk,Strongly a jerk,Mildly a jerk,Mildly a jerk,Not a jerk,Mildly a jerk,Not a jerk,Not a jerk,Mildly a jerk


In [19]:
df_all

,Time,Year,Age,Politics of Parents,Politics of Self,Spirituality,Gender,Q1: Ignored Knee Pain,Q2: Rejected Aisle Walk,Q3: Trust Fund Split 50/50,...,"Q5: Lost Cat, No Reward",Q6: Surprisig Child Drop-Off,Q7: Business Class Over Kids,"Q8: Single Parent, Four Kids",Q9: Split Spouse Child Support,"Q10: Expensive School, Cheap Career",Q11: Ex Wedding Invitation Revoked,Q12: Relatives Unapprove Wedding,"Q13: Pregnant Wife, No Drinks",Q14: Rejected Dyed Hair
0,9/23/2023 18:01:01,Junior,20,Mildly conservative,Mildly conservative,Not spiritual at all,Male,Strongly a jerk,Mildly a jerk,Strongly a jerk,...,Mildly a jerk,Mildly a jerk,Mildly a jerk,Not a jerk,Not a jerk,Strongly a jerk,Not a jerk,Not a jerk,Strongly a jerk,Mildly a jerk
1,9/23/2023 18:01:08,Junior,20,Mildly liberal,Mildly liberal,Not spiritual at all,Male,Mildly a jerk,Mildly a jerk,Strongly a jerk,...,Mildly a jerk,Not a jerk,Strongly a jerk,Not a jerk,Not a jerk,Mildly a jerk,Not a jerk,Not a jerk,Strongly a jerk,Not a jerk
2,9/23/2023 18:08:57,Junior,20,Mildly conservative,Mildly conservative,Somewhat spiritual,Male,Not a jerk,Not a jerk,Mildly a jerk,...,Not a jerk,Mildly a jerk,Mildly a jerk,Mildly a jerk,Not a jerk,Not a jerk,Not a jerk,Mildly a jerk,Strongly a jerk,Strongly a jerk
3,9/23/2023 18:09:57,Senior,22,Mildly liberal,Mildly liberal,Not spiritual at all,Female,Mildly a jerk,Not a jerk,Not a jerk,...,Not a jerk,Not a jerk,Mildly a jerk,Not a jerk,Not a jerk,Strongly a jerk,Not a jerk,Not a jerk,Not a jerk,Mildly a jerk
5,9/23/2023 18:10:13,Other,25,Mildly conservative,Mildly conservative,Not spiritual at all,Male,Mildly a jerk,Not a jerk,Not a jerk,...,Not a jerk,Not a jerk,Not a jerk,Not a jerk,Strongly a jerk,Strongly a jerk,Not a jerk,Mildly a jerk,Not a jerk,Mildly a jerk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,9/18/2024 21:34:10,Graduate Student,21,Don't know / It's complicated,Mildly liberal,Somewhat spiritual,Male,Not a jerk,Strongly a jerk,Not a jerk,...,Not a jerk,Not a jerk,Mildly a jerk,Strongly a jerk,Not a jerk,Not a jerk,Not a jerk,Mildly a jerk,Strongly a jerk,Mildly a jerk
126,9/18/2024 22:40:26,Junior,20,Neutral,Neutral,Somewhat spiritual,Male,Not a jerk,Not a jerk,Not a jerk,...,Strongly a jerk,Mildly a jerk,Mildly a jerk,Not a jerk,Not a jerk,Strongly a jerk,Strongly a jerk,Not a jerk,Strongly a jerk,Strongly a jerk
127,9/18/2024 23:06:25,Senior,21,Mildly conservative,Mildly conservative,Somewhat spiritual,Male,Not a jerk,Not a jerk,Not a jerk,...,Not a jerk,Mildly a jerk,Mildly a jerk,Not a jerk,Strongly a jerk,Mildly a jerk,Not a jerk,Mildly a jerk,Strongly a jerk,Not a jerk
128,9/18/2024 23:26:38,Graduate Student,28,Neutral,Neutral,Somewhat spiritual,Male,Not a jerk,Mildly a jerk,Mildly a jerk,...,Not a jerk,Strongly a jerk,Strongly a jerk,Mildly a jerk,Mildly a jerk,Not a jerk,Mildly a jerk,Not a jerk,Not a jerk,Mildly a jerk
